## Lines from `Galaxy` objects

Of course, you're mainly going to want to generate lines from components of a `Galaxy` (i.e. parametric or particle based stars or black holes). To do this you can utlise a component's `get_line_intrinsic` (intrinsic line emission), `get_line_screen` (line emission with a simple dust screen) or `get_line_attenuated` (line emission with more complex dust emission split into a nebular and ISM component) methods. These methods are analogous to those on a grid with the extra component specific processes, i.e. they return a `LineCollection` containing the requested lines which can either be singular, doublets, triplets or more.

In [ ]:
import synthesizer.line_ratios as line_ratios
from synthesizer.emission_models import BimodalPacmanEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emission_models.dust.emission import Blackbody
from synthesizer.grid import Grid
from synthesizer.parametric import SFH, Stars, ZDist
from unyt import Myr, kelvin

# Get a grid
grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

# Make a parametric galaxy
stellar_mass = 10**12
sfh = SFH.Constant(duration=100 * Myr)
metal_dist = ZDist.Normal(mean=0.01, sigma=0.05)
stars = Stars(
    grid.log10age,
    grid.metallicity,
    sf_hist=sfh,
    metal_dist=metal_dist,
    initial_mass=stellar_mass,
)

# Set up the emission model
model = BimodalPacmanEmission(
    grid=grid,
    tau_v_ism=0.5,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-1.3),
    dust_curve_nebular=PowerLaw(slope=-0.7),
    dust_emission_ism=Blackbody(temperature=100 * kelvin),
    dust_emission_nebular=Blackbody(temperature=30 * kelvin),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)

# To get the dust emission at a line we need to first run the
# spectra generation
stars.get_spectra(model)

# Get the lines
lines = stars.get_lines(
    (line_ratios.Hb, line_ratios.O3r, line_ratios.O3b), model
)

print(stars.lines["emergent"])

In the case of a particle based galaxy you can either get the integrated line emission...

In [ ]:
from synthesizer.emission_models import PacmanEmission
from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG

# Get the stars from a particle based galaxy
stars = load_CAMELS_IllustrisTNG(
    "../../../tests/data/",
    snap_name="camels_snap.hdf5",
    fof_name="camels_subhalo.hdf5",
    physical=True,
)[0].stars

# Set up the emission model
model = PacmanEmission(
    grid=grid,
    tau_v=0.7,
    dust_curve=PowerLaw(slope=-1.3),
    dust_emission=Blackbody(temperature=50 * kelvin),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)

# Get the spectra and lines
stars.get_spectra(model)
stars.get_lines((line_ratios.Hb, line_ratios.O3r, line_ratios.O3b), model)
print(stars.lines["emergent"])

Or per particle line emission.

In [ ]:
stars.get_particle_spectra(model)
stars.get_particle_lines(
    (line_ratios.Hb, line_ratios.O3r, line_ratios.O3b), model
)
print(stars.lines["emergent"])